Name Entity Recognition by Abrar

Importing and installing pckages

In [36]:
# Import packages
!pip install spacy
!pip install rake-nltk

# Import packages
import csv
import pandas as pd
import numpy as np
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from rake_nltk import Rake
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import spacy
from spacy import displacy
from collections import Counter
!pip install plotly
from plotly.offline import iplot
import plotly.graph_objs as go
import plotly.express as px

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asifa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asifa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading Data set

In [37]:
# Load dataset
dtypes = {'type':'category','subreddit':'category','score':'float64','title':'category','reviews':'category','author':'category'}
ds= pd.read_excel("reddit_FINAL_v12.xlsx", sheet_name="Sheet1", engine='openpyxl', dtype=dtypes)


In [38]:
# Defining Key
ds.reset_index(inplace=True)
ds.rename(columns={'index': 'key'}, inplace=True)
print(ds)

     key     type subreddit  \
0      0     post    iphone   
1      1  comment    iphone   
2      2  comment    iphone   
3      3     post    iphone   
4      4  comment    iphone   
..   ...      ...       ...   
777  777  comment    iphone   
778  778  comment    iphone   
779  779     post    iphone   
780  780  comment    iphone   
781  781  comment    iphone   

                                                 title  \
0    Iphone 14 leather cases like andar brand but i...   
1    Iphone 14 leather cases like andar brand but i...   
2    Iphone 14 leather cases like andar brand but i...   
3                     Why is my screen time incorrect?   
4                     Why is my screen time incorrect?   
..                                                 ...   
777  What profit media markt has selling iPhone low...   
778  What profit media markt has selling iPhone low...   
779                            Text tone notifications   
780                            Text tone notifi

Functions

In [39]:
# Text preprocessing
def textPreProcess(rawText, removeHTML=True, charsToRemove = r'\?|\.|\!|\;|\.|\"|\,|\(|\)|\&|\:|\-', removeNumbers=True, removeLineBreaks=False, specialCharsToRemove = r'[^\x00-\xfd]', convertToLower=True, removeConsecutiveSpaces=True):
    if type(rawText) != str:
        return rawText
    procText = rawText
        
    # Remove HTML
    if removeHTML:
        procText = BeautifulSoup(procText,'html.parser').get_text()

    # Remove punctuation and other special characters
    if len(charsToRemove)>0:
        procText = re.sub(charsToRemove,' ',procText)

    # Remove numbers
    if removeNumbers:
        procText = re.sub(r'\d+',' ',procText)

    # Remove line breaks
    if removeLineBreaks:
        procText = procText.replace('\n',' ').replace('\r', '')

    # Remove special characters
    if len(specialCharsToRemove)>0:
        procText = re.sub(specialCharsToRemove,' ',procText)

    # Normalize to lower case
    if convertToLower:
        procText = procText.lower() 

    # Replace multiple consecutive spaces with just one space
    if removeConsecutiveSpaces:
        procText = re.sub(' +', ' ', procText)

    return procText

Analysis

In [41]:
# Create a dataframe with only the description
processedReviews = pd.DataFrame(data=ds.reviews.apply(textPreProcess,charsToRemove ='', removeLineBreaks=False, removeNumbers=False).values, index=ds.index, columns=['PreProcessedText'])

C:\Users\asifa\anaconda3\lib\site-packages\bs4\__init__.py:431: MarkupResemblesLocatorWarning:

"https://iaohi.com/products/aohi-magcube-65w-pd-fast-charger" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.

C:\Users\asifa\anaconda3\lib\site-packages\bs4\__init__.py:431: MarkupResemblesLocatorWarning:

"https://preview.redd.it/yqvdvjj1r8wa1.jpeg?width=640&format=pjpg&auto=webp&v=enabled&s=10f145dc1c4c69eb3b08c6716b4ab2676564fe19" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.



In [42]:
# Remove rows with empty text
processedReviews.PreProcessedText = processedReviews.PreProcessedText.str.strip()
processedReviews = processedReviews[processedReviews.PreProcessedText != '']

In [43]:
# Load Spacy English model
nlp = spacy.load("en_core_web_sm")

In [44]:
# Check entities in review 
print(processedReviews.at[5, 'PreProcessedText'])
doc = nlp(processedReviews.at[5, 'PreProcessedText'])
print([(X.text, X.label_) for X in doc.ents])

i got this phone months ago and i have dropped a few times (well more than a few times but i can t remember) but now my phone wants to go into sos only alot and i don t know why. it never has before.
[('this phone months ago', 'DATE')]


In [45]:
# Check entities in review 
print(processedReviews.at[305, 'PreProcessedText'])
doc = nlp(processedReviews.at[305, 'PreProcessedText'])
print([(X.text, X.label_) for X in doc.ents])

exactly... he got downvoted into oblivion when he had a valid point. 

if you do not have perfect signal the photos can take an eternity to load. you go to show someone a photo just to stare at a blurry mess for what seems like anywhere from 20-30 seconds

when signal is perfect the loading is annoying but negligible. 

so yes... if you prefer to have photos with original quality directly on your device to streamline viewing without loading times then do not do this lol
[('20-30 seconds', 'TIME')]


In [46]:
# Check entities in review 
print(processedReviews.at[350, 'PreProcessedText'])
doc = nlp(processedReviews.at[350, 'PreProcessedText'])
print([(X.text, X.label_) for X in doc.ents])

i recently bought iphone 14, and am about to buy a screen protector. from the reviews i can find online, spigen is the top-rated one.

do you have this screen protector and would you recommend it? or is there any other you would recommend instead?

is this screen protector (or any other you'd recommend) compatible with apple's leather case?
[('14', 'CARDINAL')]


In [47]:
# Count the labels
labels = [x.label_ for x in doc.ents]
Counter(labels)

Counter({'CARDINAL': 1})

In [48]:
# Show top 3 labels
top_labels = [x.text for x in doc.ents]
Counter(top_labels).most_common(3)

[('14', 1)]

In [49]:
# Entities visualization
displacy.render(doc, jupyter=True, style='ent')

In [50]:
# For example, if our objective was understand what customers say about the Iphone 14 cardinal we could look for reviews that mention 14 cardinal
counter=0   # to stop after x for demostration speed
annReviews=[]
for r in processedReviews['PreProcessedText']:
  doc = nlp(r)
  for i in doc.ents:
      if i.label_=='CARDINAL':
          annReviews.append(r)
          counter = counter + 1
          break
  if counter>=3:    # Stop after the first three reviews have been found
      break

annReviews

['hold up people, my words below are subjective, you may have different opinions but my version is still worthy of your attention.\n\nok so you ve gone through my iphone journey in the title. i am currently using iphone se 2022 as my daily driver.\n\nyou may say that poor camera, small screen, smaller battery and what not. \n\nfor the camera, i am not a camera person and regarding the other two, they ve actually become a plus point for me. i never thought i would love it as much as i do now.\n\nthe screen size is so frikkin manageable, fits in the palm, single handed operation all day everyday. nostalgia hits hard and it is super comfortable in the hand. a decent back cover to complement the look and you re rocking a15 bionic perfect size beast. holding 14 series after se 22 feels bulky and tbh, i don t wanna go back anytime soon.\n\ncoming to smaller battery: previously with the bigger battery, i used to consume it all by the end of the day. would be using it for fun and games and vid